In [5]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

def get_ticker(inputs):
  ddg = DuckDuckGoSearchAPIWrapper()
  company_name = inputs["company_name"]
  return ddg.run(f"Ticker symbol of {company_name}")


functions = [
  {
    "type":"function",
    "function": {
      "name": "get_ticker",
      "description": "Given the name of a company returns its ticker symbol",
      "parameters": {
        "type": "object",
        "properties": {
          "company_name": {
            "type": "string",
            "description": "The name of the company"
          }
        },
        "required": ["company_name"]
      }
    }
  }
]

In [6]:
from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
  name="Investor Assistant",
  instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
  model="gpt-3.5-turbo-0125",
  tools=functions
)